In [1]:
require 'nokogiri'

def f node, path
  cnts = {}
  node.children
    .find_all{|x| x.class.name == "Nokogiri::XML::Element"}
    .map do |x|
        cnts[x.name] = (cnts[x.name] == nil) ? 1 : cnts[x.name] + 1
        xpath = path + "/" + x.name + "[#{cnts[x.name]}]"
        [x.name, x.children.size, xpath, f(x, xpath)]
    end
end

def nodes2text(nodes, level)
    cnts = {}
    nodes
      .map { |n|
        "#{"\t"*level}#{n[0]}, #{n[1]}, #{n[2]}" + nodes2text(n[3], level+1)
    }.reduce(""){|r, x|
        r += "\n" + x
    }
end

def diff(nodes1)
  nodes1.map{|n|
    tag_name = n[0]
    if tag_name == 'body'
      n[3]
    else
      diff(n[3])
    end
  }
end

def flatten(nodes1, r)
  nodes1.map{|n|
    r << [n[0], n[1], n[2]]
    flatten(n[3], r)
  }
  r
end


:flatten

In [2]:
html =  Nokogiri::HTML(open('sample01.html'))
nodes1 = f html, "";
text = nodes2text(nodes1, 0)
open('sample01.txt', 'w'){|f| f.puts text}

In [3]:
html =  Nokogiri::HTML(open('sample02.html'))
nodes2 = f html, "";
text = nodes2text(nodes2, 0)
open('sample02.txt', 'w'){|f| f.puts text}

In [5]:
# body tag 以下を抽出
nodes = flatten(nodes1,[]).find_all{|xs| xs[2] =~ /body/} + flatten(nodes2,[]).find_all{|xs| xs[2] =~ /body/}

# 子要素数が異なるノードを抽出
diff_nodes = nodes.group_by{|xs| xs[2]}
  .find_all{|xs|xs[1].size != 1}
  .find_all{|xs|xs[1][0][1] != xs[1][1][1]}
  .map{|xs| [xs[0], (xs[1][0][1] - xs[1][1][1]).abs]}

# メインコンテンツ(差分が最大のノード)を抽出
main_node = diff_nodes.max_by{|xs| xs[1]}
html.xpath(main_node[0] + "//img")


[#<Nokogiri::XML::Element:0x2ae925a5fe18 name="img" attributes=[#<Nokogiri::XML::Attr:0x2ae925aaa92c name="class" value="aligncenter size-medium wp-image-1521">, #<Nokogiri::XML::Attr:0x2ae925aaa904 name="src" value="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==">, #<Nokogiri::XML::Attr:0x2ae925aaa8f0 name="data-src" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0000-13-192x300.png">, #<Nokogiri::XML::Attr:0x2ae925aaa8dc name="alt" value="no-0000">, #<Nokogiri::XML::Attr:0x2ae925aaa8c8 name="width" value="192">, #<Nokogiri::XML::Attr:0x2ae925aaa8b4 name="height" value="300">, #<Nokogiri::XML::Attr:0x2ae925aaa8a0 name="srcset" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0000-13-192x300.png 192w, http://manga-fantastic.net/wp-content/uploads/2016/10/No-0000-13.png 260w">, #<Nokogiri::XML::Attr:0x2ae925aaa88c name="sizes" value="(max-width: 192px) 100vw, 192px">]>, #<Nokogiri::XML::Element:0x2ae925a5f97c name="img" attributes=[#<Nokogiri::XML::Attr:0x2ae925aa92fc name="class" value="aligncenter size-medium wp-image-1521">, #<Nokogiri::XML::Attr:0x2ae925aa92d4 name="src" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0000-13-192x300.png">, #<Nokogiri::XML::Attr:0x2ae925aa9298 name="alt" value="no-0000">, #<Nokogiri::XML::Attr:0x2ae925aa9284 name="width" value="192">, #<Nokogiri::XML::Attr:0x2ae925aa925c name="height" value="300">, #<Nokogiri::XML::Attr:0x2ae925aa9248 name="srcset" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0000-13-192x300.png 192w, http://manga-fantastic.net/wp-content/uploads/2016/10/No-0000-13.png 260w">, #<Nokogiri::XML::Attr:0x2ae925aa9234 name="sizes" value="(max-width: 192px) 100vw, 192px">]>, #<Nokogiri::XML::Element:0x2ae925a5dd0c name="img" attributes=[#<Nokogiri::XML::Attr:0x2ae925aa7e0c name="class" value="alignnone size-medium wp-image-1547">, #<Nokogiri::XML::Attr:0x2ae925aa7d30 name="src" value="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==">, #<Nokogiri::XML::Attr:0x2ae925aa7ce0 name="data-src" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0014-11-191x300.png">, #<Nokogiri::XML::Attr:0x2ae925aa7ccc name="alt" value="no-0014">, #<Nokogiri::XML::Attr:0x2ae925aa7cb8 name="width" value="191">, #<Nokogiri::XML::Attr:0x2ae925aa7ca4 name="height" value="300">, #<Nokogiri::XML::Attr:0x2ae925aa7c68 name="srcset" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0014-11-191x300.png 191w, http://manga-fantastic.net/wp-content/uploads/2016/10/No-0014-11.png 260w">, #<Nokogiri::XML::Attr:0x2ae925aa7c40 name="sizes" value="(max-width: 191px) 100vw, 191px">]>, #<Nokogiri::XML::Element:0x2ae925a5d58c name="img" attributes=[#<Nokogiri::XML::Attr:0x2ae925aa6520 name="class" value="alignnone size-medium wp-image-1547">, #<Nokogiri::XML::Attr:0x2ae925aa650c name="src" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0014-11-191x300.png">, #<Nokogiri::XML::Attr:0x2ae925aa64f8 name="alt" value="no-0014">, #<Nokogiri::XML::Attr:0x2ae925aa64e4 name="width" value="191">, #<Nokogiri::XML::Attr:0x2ae925aa64d0 name="height" value="300">, #<Nokogiri::XML::Attr:0x2ae925aa64bc name="srcset" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0014-11-191x300.png 191w, http://manga-fantastic.net/wp-content/uploads/2016/10/No-0014-11.png 260w">, #<Nokogiri::XML::Attr:0x2ae925aa64a8 name="sizes" value="(max-width: 191px) 100vw, 191px">]>, #<Nokogiri::XML::Element:0x2ae925a5bc78 name="img" attributes=[#<Nokogiri::XML::Attr:0x2ae925aa2a9c name="class" value="alignnone size-medium wp-image-1548">, #<Nokogiri::XML::Attr:0x2ae925aa2a74 name="src" value="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==">, #<Nokogiri::XML::Attr:0x2ae925aa2a4c name="data-src" value="http://manga-fantastic.net/wp-content/uploads/2016/10/No-0015-10-178x300.png">, #<Nokogiri::XML::Attr:0x2ae925aa2a38 name="alt" value="no-0015">, #<Nokogiri::XML::